In [2]:
import json
import random
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.graph import StateGraph, MessagesState, START, END, MessageGraph
from IPython.display import display, Image
from typing import TypedDict, Annotated, Literal
from pydantic import BaseModel, Field
import operator
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode

load_dotenv("../.env")
llm = ChatGroq(
    # model="llama3-groq-70b-8192-tool-use-preview",
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
#embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

memory = MemorySaver()
config = {"configurable": {"thread_id": 1}}

In [3]:
# Import tools from tools.py
from tools import get_current_weather, get_system_time

In [4]:
model_with_tools = llm.bind_tools(
    tools=[get_current_weather, get_system_time],
)

tool_mapping = {
    'get_current_weather': get_current_weather,
    'get_system_time': get_system_time,
}

In [5]:
# Define prompt template
prompt = PromptTemplate(
    template=
    """
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
        You are a smart Agent. 
        You are a master at understanding what a customer wants and utilize available tools only if you have to.
    <|eot_id|>
    <|start_header_id|>user<|end_header_id|>
        Conduct a comprehensive analysis of the request provided. \n
        USER REQUEST:\n\n {initial_request} \n\n

    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_request"],
)

agent_request_generator = prompt | model_with_tools

In [6]:
result = agent_request_generator.invoke(
    {"initial_request": "What is the weather in woodbury in MN?"}
    )
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_nc4k', 'function': {'arguments': '{"location": "Woodbury, MN"}', 'name': 'get_current_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 833, 'total_tokens': 851, 'completion_time': 0.072, 'prompt_time': 0.160661552, 'queue_time': 0.14008964799999998, 'total_time': 0.232661552}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-003294fc-1a4e-42c5-81e5-f2b81ea04949-0' tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'Woodbury, MN'}, 'id': 'call_nc4k', 'type': 'tool_call'}] usage_metadata={'input_tokens': 833, 'output_tokens': 18, 'total_tokens': 851}


In [9]:

# Extract the last AI message from messages
last_ai_message = None
if isinstance(result, AIMessage):
    last_ai_message = result

#print("Last AI Message:", last_ai_message)

if last_ai_message and hasattr(last_ai_message, 'tool_calls'):
    print("last_ai_message:", last_ai_message)
    print("tool name :: ", last_ai_message.tool_calls[-1]["name"])
    print("tool args :: ", last_ai_message.tool_calls[-1]["args"])
else:
    print("No tool calls found.")


last_ai_message: content='' additional_kwargs={'tool_calls': [{'id': 'call_nc4k', 'function': {'arguments': '{"location": "Woodbury, MN"}', 'name': 'get_current_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 833, 'total_tokens': 851, 'completion_time': 0.072, 'prompt_time': 0.160661552, 'queue_time': 0.14008964799999998, 'total_time': 0.232661552}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-003294fc-1a4e-42c5-81e5-f2b81ea04949-0' tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'Woodbury, MN'}, 'id': 'call_nc4k', 'type': 'tool_call'}] usage_metadata={'input_tokens': 833, 'output_tokens': 18, 'total_tokens': 851}
tool name ::  get_current_weather
tool args ::  {'location': 'Woodbury, MN'}


In [8]:
#run tool
for tool_call in result.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    print(tool_output)

Sunny, 78F
